## Cross-Validation with Symmetric Pair-Input Data

In the symmetric Pair-Input Data,  data instance shares common pair-members, which  result the strong dependencies among data samples and this violates the IID assumption. Thus, the modfied verison of cross-validaiton for symmetric Pair-Input Data is applied in this task.  The c-index is used as the evaluation metric.

In such data, three possible types of pairs with respect to training set can be found:
      
      A – both members are present in training set
      B – only one member is present in training set
      C – neither member is present in training set



                        =====  Implementation start  ======

Note: All the three cross-validations are perfomed with normalized data and with out normalized. The z-score is used for normalization. 



In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import operator


In [2]:
# load the dataset
def load_dataset(data):
    df = pd.read_csv(data, header=None) # read dataset
    return df
    

In [3]:
#run load_dataset function
df = load_dataset('Dataset/features.data')
print('Dimension of dataframe:', df.shape)

#check data types 
print('\ndata types:',df.dtypes)
df.head()

Dimension of dataframe: (95, 41)

data types: 0     int64
1     int64
2     int64
3     int64
4     int64
5     int64
6     int64
7     int64
8     int64
9     int64
10    int64
11    int64
12    int64
13    int64
14    int64
15    int64
16    int64
17    int64
18    int64
19    int64
20    int64
21    int64
22    int64
23    int64
24    int64
25    int64
26    int64
27    int64
28    int64
29    int64
30    int64
31    int64
32    int64
33    int64
34    int64
35    int64
36    int64
37    int64
38    int64
39    int64
40    int64
dtype: object


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,94,0,16,51,54,54,74,21,43,61,...,52,26,55,47,225,140,43,30,132,92
1,21,0,6,8,12,5,19,5,6,20,...,6,8,10,7,32,23,11,7,26,19
2,88,0,14,59,58,55,76,24,53,57,...,57,32,56,52,227,146,45,35,135,105
3,52,0,5,18,17,51,47,16,50,15,...,19,10,21,19,184,109,15,10,106,75
4,39,0,15,27,23,51,64,19,65,15,...,25,16,31,24,194,118,16,15,117,106


                             ==== Normalize data with z-score ====
    
Since every input feature is of 'int' data type, it should be change into 'float'  because when you try to convert them into normalized form it will  just 'int' type. So, the numbers like '1.23' and '1'  will be 1, though they are not the same. This will cause the difference in result then as it acutally should be.

In [4]:
# Normalize the data
data_matrix = df.values# change to numpy array
data_matrix = data_matrix.astype(float) # change to float type

for i in range(len(data_matrix.T)): # transpose and calcualte 'mean' and 'stdv' row-wise
    
    # check if all elements in (ith) row are zeros 
    #  then  calculate 'mean' and 'stdv' for  other than ith rows
    if np.all(data_matrix.T[i] == 0) == False:
        mean = np.mean(data_matrix.T[i])
        stdv = np.std(data_matrix.T[i])
        
        for j in range(len(data_matrix)) : # apply z-score in each element of df
            data_matrix[j][i]= (data_matrix[j][i] - mean)/stdv
        
data_matrix = pd.DataFrame(data_matrix)  # change to pandas dataframe 
data_matrix.head()




,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1.800051,0.0,1.669472,1.937529,1.850945,0.820203,1.182565,0.591083,0.133767,2.191697,...,1.897792,1.666761,1.977547,1.811226,0.936060,1.042499,2.033321,1.904285,0.950928,0.863910
1,-1.146221,0.0,-0.500156,-1.157820,-1.056642,-1.507873,-1.086588,-1.537936,-1.739502,-0.204895,...,-1.451704,-1.039945,-1.459044,-1.520167,-1.677808,-1.608468,-0.836361,-0.930090,-1.673310,-1.676895
2,1.557892,0.0,1.235546,2.513408,2.127858,0.867715,1.265080,0.990274,0.640056,1.957883,...,2.261868,2.568996,2.053916,2.227650,0.963147,1.178446,2.212676,2.520454,1.025199,1.316383
3,0.104936,0.0,-0.717119,-0.437971,-0.710501,0.677668,0.068617,-0.074236,0.488169,-0.497162,...,-0.505107,-0.739200,-0.618988,-0.520749,0.380782,0.340106,-0.477651,-0.560389,0.307247,0.272216
4,-0.419743,0.0,1.452509,0.209893,-0.295131,0.677668,0.769992,0.324955,1.247602,-0.497162,...,-0.068216,0.163036,0.144699,-0.104325,0.516216,0.544027,-0.387973,0.055779,0.579574,1.351188


                    === target variable(label) is combined with input feature ====
 Since Knn algorithm is desinged in such a way that it takes  input features plus target variable(labels) together as the data matrix.

In [5]:
# target variable(output)
df_target = pd.read_csv('Dataset/labels.data', names=['Target'])
data_matrix_target = pd.concat([data_matrix, df_target], axis=1)
data_matrix_target.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Target
0,1.800051,0.0,1.669472,1.937529,1.850945,0.820203,1.182565,0.591083,0.133767,2.191697,...,1.666761,1.977547,1.811226,0.936060,1.042499,2.033321,1.904285,0.950928,0.863910,1
1,-1.146221,0.0,-0.500156,-1.157820,-1.056642,-1.507873,-1.086588,-1.537936,-1.739502,-0.204895,...,-1.039945,-1.459044,-1.520167,-1.677808,-1.608468,-0.836361,-0.930090,-1.673310,-1.676895,1
2,1.557892,0.0,1.235546,2.513408,2.127858,0.867715,1.265080,0.990274,0.640056,1.957883,...,2.568996,2.053916,2.227650,0.963147,1.178446,2.212676,2.520454,1.025199,1.316383,0
3,0.104936,0.0,-0.717119,-0.437971,-0.710501,0.677668,0.068617,-0.074236,0.488169,-0.497162,...,-0.739200,-0.618988,-0.520749,0.380782,0.340106,-0.477651,-0.560389,0.307247,0.272216,1
4,-0.419743,0.0,1.452509,0.209893,-0.295131,0.677668,0.769992,0.324955,1.247602,-0.497162,...,0.163036,0.144699,-0.104325,0.516216,0.544027,-0.387973,0.055779,0.579574,1.351188,1


In [6]:
 '''==== data with out normalization ====='''
    
# comibine input features with target class/label
data_matrix_without_norm= pd.concat([df, df_target], axis=1)
data_matrix_without_norm.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,Target
0,94,0,16,51,54,54,74,21,43,61,...,26,55,47,225,140,43,30,132,92,1
1,21,0,6,8,12,5,19,5,6,20,...,8,10,7,32,23,11,7,26,19,1
2,88,0,14,59,58,55,76,24,53,57,...,32,56,52,227,146,45,35,135,105,0
3,52,0,5,18,17,51,47,16,50,15,...,10,21,19,184,109,15,10,106,75,1
4,39,0,15,27,23,51,64,19,65,15,...,16,31,24,194,118,16,15,117,106,1


In [7]:
# read pairs-input in pandas dataframe
input_pair = pd.read_csv('Dataset/pairs.data', header=None)
input_pair.head()

,0,1
0,P0,P9
1,P6,P19
2,P0,P4
3,P5,P17
4,P15,P17


                    ===  Knn from scratch implementation ====

The three functions are used to perfome Knn classifier. The eculidan_dist() function calculates the eculidan distance between the two instances of data. The second function get_neighbours() finds the 'k' neighbours from the traiing dataset for the test instance, and the third function get_response() finds the 'class' for test instance  that belongs to higest votes with in the 'k' neigbours.

In [8]:
#calculates eculidan distance between two data instances
def eculidan_dist(data1, data2):
    
    distance = 0
    length = len(data2) -1 # since last variable is target 
        
    # loop through all  input variables (except target variables)
    for x in range(length):
        distance += np.power((data1[x]- data2[x]),2)
    return np.sqrt(distance)
    


In [9]:
# finds k nearest  neighbors of  test instance 
def get_neighbours(trainingSet, testInstance, k):
        distance = []
        
        # find distance  between  a test instance to all training instances
        for x in range(len(trainingSet)):
            dist = eculidan_dist(trainingSet[x], testInstance) # call eculidan_dist function
            distance.append((trainingSet[x], dist)) 
        
        # sort the distance in ascending order
        distance.sort(key=operator.itemgetter(1))
        

        #store the  k neighbours
        neighbours = []
        for x in  range(k):
            # takes only neighbours from distance list and append 
            neighbours.append(distance[x][0]) 
        return neighbours

In [10]:
# finds class that test instance belongs to. This is done by finding the  the most frequent class in the 'k' neigbours
def get_response(neighbors):
    classVotes = {} # for counting each class/label
    
    #counts each class in 'k' neighbours
    for x in range(len(neighbors)):
        response = neighbors[x][-1] # take last attribute(target variable) 
        if response in classVotes:
            classVotes[response] +=1
            
        else:
            classVotes[response] = 1
    
    # sorts classes with higest values
    sortedVotes = sorted(classVotes.items(),key=operator.itemgetter(1), reverse=True) # appear as like [('b', 3), ('a', 2)]
    return  sortedVotes[0][0] # take first neighbour(of higest votes)

   ==== C-index Implementation start here =====

In [11]:
#implementation  of C_index
def c_index (true_labels, predictions):
    n = 0
    h_sum =0
    
    for i in range(len(true_labels)):
        t = true_labels[i]
        p = predictions[i]
        
        
        for j in range(i+1, len(true_labels)):
            nt = true_labels[j]
            np = predictions[j]
            
            if( t!=nt):
                n += 1
                
                if(p < np and t <  nt) or (p > np and t > nt):
                    h_sum += 1
                
                elif(p == np):
                    h_sum += 0.5
                
    c_index = h_sum/n
    return c_index

In [12]:
# testing c-index implementation
true_labels = [-1, 1, 1, -1, 1]
predictions = [0.60, 0.80, 0.75, 0.75, 0.70]
c_index(true_labels, predictions)

0.75

                   ==== Leave one out cross_validation ====
  This is the standard leave one out cross validaiton (with out modification). Each sample in the dataset becomes as test and remainig samples  become training set. So, the repeatation of this step (testing) goes to 'n' time. 'n' refers to total number of samples. This method belong to 'case-A' since the training set consists of both shared members of instance data.

In [13]:
def lev_one_out_cv(dataset, input_pair):
    '''
    dataset = dataset(input features + labels) in numpy array
    input_pair = pair of protein  for input features in numpy array
    
    '''
    
    
    predicted_label = [] 
    true_label = []
    
    
    # ====split data for test and train ===
   
    # use index for input-pair and dataset
    for x in range(len(dataset)):
        test_instance = dataset[x] # feature matrix for test instance of 'x' index
        train_data = np.concatenate([dataset[0:x],dataset[1+x:]]) # train_data (other than test instance)
        true_label.append(test_instance[-1]) # append class of test_instance
          
        
       
        
    
        '''=== Knn start from here ==='''
        neighbours = get_neighbours(train_data, test_instance, 1) # call get_neighbours()
        predict = get_response(neighbours) # call get_response()
        predicted_label.append(predict)
        
    
    ''' ==== compute c_index  ==== '''
    c_index_val = c_index(true_label, predicted_label) # call c_index()
        
    return c_index_val
            
    

In [14]:
#run cross validataion function
c_val= lev_one_out_cv(data_matrix_without_norm.values, input_pair.values)# with out normalization 
print('C_index with out data normalization: {0:.3f}'.format(c_val))

c_val_1= lev_one_out_cv(data_matrix_target.values, input_pair.values) #normalized data
print('C_index with data normalization: {0:.3f}'.format(c_val_1))



C_index with out data normalization: 0.762
C_index with data normalization: 0.692


                     ==== Modified leave-one-out Cross validation ==== 
In this crossvalidation method, two for-loop is used. First loop is used to select 'test instance' and second loop is used for training set, which  removes the both members shared by 'test instance'. This cross-validation method belongs to the 'case-C' since in training set, there is no shared members at all.


In [15]:
def cross_validation(dataset, input_pair):
    '''
    dataset = dataset(input features + labels) in numpy array
    input_pair = pair of protein  for input features in numpy array
    
    '''
    
    
    predicted_class = [] # store predicted class
    true_class = [] # store actual class of test instance
    
    
    ''' split data for test and train '''
    # iterates over data samples
    for x in range(len(dataset)):# for test instance
        '''x= index for numpy array'''
        
        test_instance = dataset[x] #test instance of 'x' index
        train_data = [] # store train_data
        true_class.append(test_instance[-1]) # append class/labels of test_instance
    
        
        #compare 'input-pair' of test_instance with rest of data_points in dataset
        #Remove those which are common one from training set
        for y in range(len(dataset)):
            if y!= x: # other then test instance index
                if input_pair[y][0] != input_pair[x][0] and input_pair[y][0]!= input_pair[x][1] and input_pair[y][1]!= input_pair[x][0] and input_pair[y][1]!= input_pair[x][1]:
                     train_data.append(dataset[y])
                
        
        '''=== Knn start from here ==='''
        train_dataset = np.array(train_data)
        neighbours = get_neighbours(train_dataset, test_instance, 1) # call get_neighbours()
        predict = get_response(neighbours) # call get_response()
        predicted_class.append(predict)
        
    
    
    ''' ==== compute c_index  ==== '''
    c_index_val = c_index(true_class, predicted_class) # call c_index()
        
    return c_index_val
            
            
    

In [16]:
#run cross validataion function
c_val= cross_validation(data_matrix_without_norm.values, input_pair.values) # with out normalized data
print('C_index with out data normalization: {0:.3f}'.format(c_val))

c_val_1= cross_validation(data_matrix_target.values, input_pair.values) #nomarlised data
print('C_index with data normalization: {0:.3f}'.format(c_val_1))


C_index with out data normalization: 0.631
C_index with data normalization: 0.673


### Observation:
1.Normal leave_one_out crossvalidation produces greater c_index than modified cross_validation. The reason behind this is in 'normal leave_one_out' cv  there is information about test instance in training set. Thus, leakage of information  about test instance is happend in normal leave_one_out. On the other hand, in modified cv there is no information leakege of test instance since we have removed all the shared members of test instance in training set.

2.Unmodified cross_validation is suitable for case_A where all the two shared members of test instance are found in training set, while modified cross_validaiton is suitable for case_C since there is no shared member of test instance in trainig set

                                ==== Cross_validation for case_B =====
In this task, the cross_validation suitable for the 'case_B'(=only one member is in training set) is designed.
The previous cross_validation methods are not appropriate for the 'case_B' since unmodified crossvalidation method has 2 shared members of test instance in training set. Likewise, in the previous modified cross-validation there is no shared members in the trainig set.  

In this cross-validation, the training set should  have only one(out of two) member of test instance. This can be done by predicting the same 'test instance' two time ( first-time:  trianing dataset contains only first member and second-time: the training dataset contains only second member or vice versa )


For example, if S(a,b), 'S' is the test instance belongs to (a,b) pairs then, this same test should be tested/predicted in two times with two difference training dataset, which is described as below:
   
      In the first prediction, the train data contains first member i.e. a (lets say)
      In second prediction,the train data contains second member i.e. b (lets say)

In [43]:
def cross_validation_b(dataset, input_pair):
    '''
    dataset = dataset(input features + labels) in numpy array
    input_pair = pair of protein  for input features in numpy array
    
    '''
    
    
    predicted_class = [] 
    true_class = []
    
    
    # ====split data for test and train ===
   
    # use index for input-pair and dataset
    for x in range(len(dataset)):# for test instance 
        test_instance = dataset[x] #test instance of 'x' index
        train_data_1 = [] #train_data that includes first member of test instance
        train_data_2 = [] # store train_data hat includes second member of test instance
        true_class.append(test_instance[-1]) # append class of test_instance
        true_class.append(test_instance[-1]) # append class of test_instance
    
        #print(input_pair[x][0], input_pair[x][1])
        
        #compare 'input-pair' of test_instance with rest of data_points in dataset
        #train data should have one  member
        
        for y in range(len(dataset)):
            if y!= x:  # other than test instance
                
                # compeltly no shared members
                if input_pair[y][0] != input_pair[x][0] and input_pair[y][0]!= input_pair[x][1] and input_pair[y][1]!= input_pair[x][0] and input_pair[y][1]!= input_pair[x][1]:
                     train_data_1.append(dataset[y])
                    #train_data.append((input_pair[y][0],input_pair[y][1]))
                        
                 
                # take only one member (first member) in training dataset
                elif input_pair[y][0] == input_pair[x][0] or input_pair[y][1]== input_pair[x][0]: 
                    
                    train_data_1.append(dataset[y])
                    
                
        
        
        '''=== Knn start from here ==='''
        train_dataset = np.array(train_data_1)
        neighbours = get_neighbours(train_dataset, test_instance, 1) # call get_neighbours()
        predict = get_response(neighbours) # call get_response()
        predicted_class.append(predict)
        
        for y in range(len(dataset)):
            if y!= x:  # other than test instance
                
                # compeltly no shared members
                if input_pair[y][0] != input_pair[x][0] and input_pair[y][0]!= input_pair[x][1] and input_pair[y][1]!= input_pair[x][0] and input_pair[y][1]!= input_pair[x][1]:
                     train_data_2.append(dataset[y])
                    #train_data.append((input_pair[y][0],input_pair[y][1]))
                        
                    
                # take only one member(second member) in training dataset
                elif input_pair[y][0] == input_pair[x][1] or input_pair[y][1]== input_pair[x][1]: 
                    
                    train_data_2.append(dataset[y])
         
  
        '''=== Knn start from here ==='''
        train_dataset = np.array(train_data_2)
        neighbours = get_neighbours(train_dataset, test_instance, 1) # call get_neighbours()
        predict = get_response(neighbours) # call get_response()
        predicted_class.append(predict)
        
    
    
    ''' ==== compute c_index  ==== '''
    c_index_val = c_index(true_class, predicted_class) # call c_index()
        
    return c_index_val
            
            
    

In [44]:
#run cross validataion function
c_val= cross_validation_b(data_matrix_without_norm.values, input_pair.values) # with out normalized data
print('C_index with out data normalization: {0:.3f}'.format(c_val))

c_val_1= cross_validation_b(data_matrix_target.values, input_pair.values) #nomarlised data
print('C_index with data normalization: {0:.3f}'.format(c_val_1))

C_index with out data normalization: 0.697
C_index with data normalization: 0.690


### Observation
The c_index(0.697) obtained from third cross-validation(case-B) is beteween the unmodifed cross-validation(case-A) c-index (0.762) and  first modified cross-validation(case-C) c-index(0.631). The trainig set from this cross_validataion  still consists some information (one member) of test instance. Thus, c-index is greter than the previous modified cv version, which consits no infroamtion about test instance.  And, this c_index is less than the c-index from unmodified cross-validation because the infomation about test instance in training set from unmodified cross-validation has more( consist two members of test instance).


